# Solving the Schrodinger Equation for a particle in a box for the ground state eigenfunctions using numerical methods

## Janos Revesz, SN: 19111202
The notebook contains solving the Time Independent Schrödinger Equation of a particle in a box, as a boundary value problem, using numerical methods and comparing these results to the analytical solution. The TISE:
$$-\frac{1}{2} \frac {d^2}{d x^2} \psi(x) = E \psi(x)$$
  
The units used are atomic units, so $m=\hbar=1$ where m is the mass of the electron. Bohr radius is used for distances and Hartrees for energies. We only solve for the ground state solution.;

In [ ]:
# Appropriate imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

### 1. 
Separating the 2nd oreder TISE into two linked 1st order differential equations:
$$ \frac {d^2}{d x^2} \psi(x) = -2 E \psi(x) $$
   
  $$ \frac {d}{d x} \psi(x) = \phi(x)$$ and $$  \frac {d}{d x} \phi(x)=-2 E \psi(x)$$
then implementing the right hand sides in the following cell as a function.

In [ ]:
def RHS_Schro(y,E):
    """Implementation of the RHS of the Schrodinger equation
    
    Input:
    y:  input values (two component array), first element is the values for psi
    the second element is values for phi
    E:  Energy, which can be seen as a constant
    Output:
    Differentials of psi and phi(two component array)
    """
    # Separate input
    psi = y[0]
    phi = y[1]
    # Calculate differentials
    dpsi = phi
    dphi = -2*E*psi
    return np.array((dpsi,dphi))

### 2. Runge-Kutta method implementation for 2nd order ODEs
The R-K methods use intermediate gradients to improve accuracy. The fourth order R-K method is accurate up to dx^4 . It evaluates y(x+dx) first the same way as Euler's method then adds three corrections. Two of these corrections are evaluated at mid points and the third at the end-point. Because RK4 is accurate up to dx^4 we can use quite large step sizes to solce ODEs.

In [ ]:
def RK4_solver(fun,y0,dx,E,N):
    """Solve dy/dt = fun(y,t) using fourth-order RK method.
    Inputs:
    fun  f(y,t)
    y0   Initial condition - assumed to be two-component
    dx   Spacing in x
    E    Parameter to pass to fun
    N    Number of steps
    Returns: two arrays of length N+1 (x and v or equivalent)
    """
    # Creating arrays for psi and phi gridpoint
    psi = np.zeros(N+1)
    phi = np.zeros(N+1)
    # Initializing first components of arrays using initial conditions
    psi[0] = y0[0]
    phi[0] = y0[1]
    x = 0
    y = y0
    # Evaluating psi and phi at the gridpoints using the Runge-Kutta method
    # up to 4th order
    for i in range(N):
        k1 = dx*fun(y,E)
        k2 = dx*fun(y+0.5*k1,E)
        k3 = dx*fun(y+0.5*k2,E)
        k4 = dx*fun(y+k3,E)
        y=y+(k1+2*k2+2*k3+k4)/6
        psi[i+1] = y[0]
        phi[i+1] = y[1]
    # Return psi and phi evaluated from x to x+N*dx
    return psi, phi

### 3. Testing the numerical solution for E=1

Now testing the RK4 method to find the eigenstates of the particle in a box. We haven't used the second boundary condition psi(1)=0 so the wavefuntion doesn't fo to 0 as x goes to 1.

In [ ]:
# Setting up gridpoints for x
dx = 0.01
total_x = 1.0
N = int(total_x/dx)

# Choose lower limit
E0 = 1.0
# Create a two-component array with values at x=0
psi0 = np.array([0.0,1.0])

# Numerical solution of TISE using RK4
psi, phi = RK4_solver(RHS_Schro,psi0,dx,E0,N)
# Setting up grid for plotting
x = np.linspace(0,total_x,N+1)

# Plotting the numerical solution
plt.plot(x,psi,label="E = 1.0")
plt.xlabel("x (Bohr)",fontsize=15)
plt.ylabel(r"$\psi(x)$",fontsize=15)
plt.title("Numerical solutin for TISE when E=1")
plt.legend()
plt.grid()
P0 = psi[-1]
print("The numerical solution at x=1 Bohr is ",P0)

### 4. Using the bisection method to find E where $ \psi(1)=0$
Now using the boundary condition psi(1)=0 we find the energy of the ground solution to the TISE. We do this by setting x=1 and evaluating psi(x) for different energies. Using the bisection method we can find the energy for which psi(1) is 0. First I plot psi(1) against E so I can apply the bisection method by setting the upper and lower limit.

In [ ]:
# PLOTTING PSI(1) AGAINST E

# grid for E from 1 to 11
E = np.arange(1,11.01,0.01)

# define a function that evaluates psi at x=1 for different energies
def get_psi(E):
    """get_psi returns psi evaluated at x=1 for different energies
    
    E = the energy where psi(1) is evaluated
    
    return: psi(1) evaluated at E
    """
    psi, phi = RK4_solver(RHS_Schro,psi0,dx,E,N)
    return psi[-1]

# creating a grid for the y axes by evaluating psi(1) for 1<=E<=11
psi_1 = np.zeros(1001)
for i in range(1001):
    psi_1[i] = get_psi(E[i])
    
# Plotting psi(1) for different energies
plt.plot(E,psi_1)
plt.xlabel("E (Hartree)",fontsize=12)
plt.ylabel(r"$\psi(1)$",fontsize=12)
plt.title("Analytical solution of TISE evaluated at x=1 for 1 < E < 11")
plt.grid()

In [ ]:
# Using the bisection method to find where psi(1) = 0

# Setting the tolerance 
# Note: We need a very low tolerance to have meaningfull error at Q6
tol = 1e-10
# Lower limit
E0 = 1.0
P0 = get_psi(E0)
# Upper limit
E1 = 11.0
P1 = get_psi(E1)
# Counter
n=0
# Change the lower or upper limit to the middle point
while abs(E0-E1)>tol:
    # Bisection
    n += 1
    E2 = (E0+E1)/2
    P2 = get_psi(E2)
    # Evaluate psi for Emid
    if P2*P1 < 0.0:
        E0 = E2
        P0 = P2
    else:
        E1 = E2
        P1 = P2

print("Finished after ",n," iterations with root at ",E1, " value is ",P1)

### 5. Normalising and plotting the numerical solution of TISE
Now that we have a wave function that behaves as the boundary conditions require we can normalize it. psi^2 is the probabilty density of the particle being at x so integrated form 0 to 1 psi^2 must be zero.
$$\int_{0}^{1} |\psi(x)|^2 dx = A$$
  
  the normalized wavefunciton then:
  $$\psi(x)_{norm} = \frac {1}{\sqrt{A}} \psi(x) $$

In [ ]:
# the ground state solution of the wavefunction 
psi_num, phi = RK4_solver(RHS_Schro,psi0,dx,E1,N)

# Potting the normalized and not normalized wave functions
x = np.arange(0,total_x+0.01,dx)
plt.plot(x,psi_num,label="not normalized")
plt.xlabel("x (Bohr)",fontsize=15)
plt.ylabel(r"$\psi(x)$",fontsize=15)
plt.title("Numerical solutin for TISE when E=4.93")
norm = 1/np.sqrt(integrate.simps(psi_num**2,x))
print("The normalizaton factor is: ",norm)
psi_num = psi_num*norm
plt.plot(x,psi_num,label="normalized")
plt.legend()
plt.grid()

### 6. Compare to analytic wavefunction
The ground state solution of the wave function for a particle in a box is well known. We can implement the analytical solution to find how accurate our numerical solution was.
  The analytical solution is:
   $$ \psi(x) = \frac {1}{\sqrt{A}} sin(\pi x) $$
     
We normalize the analytical solution the same way we normalized the numerical solution.

In [ ]:
# Create the analytical wave function
k = np.pi
psi_ana = np.sin(k*x)
# Normalize the analytical wave function
norm = 1/np.sqrt(integrate.simps(psi_ana**2,x,0.01))
psi_ana = psi_ana*norm

# Plotting the difference of the two wave functions
plt.plot(x,(psi_ana-psi_num))
plt.xlabel("x (Bohr)", fontsize=15)
plt.ylabel(r"|$\psi_{anal}(x)-\psi_{num}(x)$|",fontsize = 13)
plt.grid()

Comment: to get a meaningful plot for the error the tolerance of the bisection method has to be 1e-10 or lower. That is because the error is on the scale of 1e-10 so any larger tolerance would diguise it.

Brief conclusions or commentary

The numerical solution is accurate up to 1e-10 with appropriately set tolerances for the bisection method. The RK4 method should provide an accuracy for dx=0.01 of dx^4= 1e-8. It seems like the accuracy is larger in practice for our case after normalization. The not normalized wave function is wastly different from the normalized one and the analytical one and this doesn't get better for dx=0.001.